### Lab 3:  Variables and Templates


[From Ansible doc](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

Using Variables
Ansible uses variables to manage differences between systems. With Ansible, you can execute tasks and playbooks on multiple different systems with a single command. To represent the variations among those different systems, you can create variables with standard YAML syntax, including lists and dictionaries. You can define these variables in your playbooks, in your inventory, in re-usable files or roles, or at the command line. You can also create variables during a playbook run by registering the return value or values of a task as a new variable.

After you create variables, either by defining them in a file, passing them at the command line, or registering the return value or values of a task as a new variable, you can use those variables in module arguments, in conditional “when” statements, in templates, and in loops. [The ansible-examples github repository](https://github.com/ansible/ansible-examples) contains many examples of using variables in Ansible.

Once you understand the concepts and examples on this page, read about Ansible facts, which are variables you retrieve from remote systems. With Ansible you can retrieve or discover certain variables containing information about your remote systems or about Ansible itself. Variables related to remote systems are called facts. With facts, you can use the behavior or state of one system as configuration on other systems. For example, you can use the IP address of one system as a configuration value on another system. Variables related to Ansible are called magic variables.

## Creating valid variable names
Not all strings are valid Ansible variable names. A variable name can only include letters, numbers, and underscores. Python keywords or playbook keywords are not valid variable names. A variable name cannot begin with a number.

Variable names can begin with an underscore. In many programming languages, variables that begin with an underscore are private. This is not true in Ansible. Variables that begin with an underscore are treated exactly the same as any other variable. Do not rely on this convention for privacy or security.

[Defining Variables](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#define-variables-in-inventory)

## Variables

For this lab, we will leverage simple variables to be later used in a task. The task will append some content to a file and substitute the relevant variables.

### Start by defining the variable files:
Update the relevant field in the variables_local.yml file definition by editing the  next cell

In [ ]:
# Define the local and target variable files:

cat > vars/variables_local.yml <<EOF
LABSTUDENTNAME: "INPUT YOUR NAME HERE"
#LABSTUDENTID: "{{ STDID }}"
LABSTUDENTID: "12"
LABLOCATION:  "INPUT YOUR COUNTRY"
EOF

cat > vars/variables_target.yml <<EOF
LABSTUDENTNAME: "APOLLO"
LABSTUDENTID: "11"
LABLOCATION: "THE MOON"
EOF

#### Now create the task

In [ ]:
# Define an inventory:

cat > inventory <<EOF
[target]
{{ IP-WKSHP-Ansible101 }}

[local]
localhost
EOF

# Define a play

cat > play1.yml << EOF
- hosts: localhost
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
    - name: Adapt lab3.1.txt file
      blockinfile:
        create: yes
        path: /student/student{{ STDID }}/lab3.1.txt
        block: |
          ##############################################################################################################################################################################
          # This file was generated by the Ansible101 HPE DEV Worshop-on-Demand lab3 for {{ LABSTUDENTNAME }} with id {{ LABSTUDENTID }}  running from {{ hostvars[inventory_hostname]['ansible_facts']['nodename'] }}
          ##############################################################################################################################################################################
EOF

Now let's execute the playbook on the inventory file we have just created.

You will note that for now, the task is targetting the localhost only. The point is to show that the principle here.

In [ ]:
ansible-playbook -i inventory play1.yml

### Now take a look at the result

In [ ]:
cat /student/student{{ STDID }}/lab3.1.txt

## Templates

in this part, we will create a template file (jinja Template with j2 extension) that will be used to create the final file.

We will then introduce conditions to alter the content of the file depending on the target. We will use the [ansible template module](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/template_module.html) to achieve this



In [ ]:
# Define a play

cat > play2.yml << EOF
- hosts: all
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
    - name: Template a file to lab3.2.txt
      template:
        src: templates/lab3.2.txt.j2
        dest: /tmp/lab3.2.txt
        mode: '0644'
EOF

### Execute the play now

In [ ]:
ansible-playbook -i inventory play2.yml

### Let's look at the results

In [ ]:
ssh localhost cat /tmp/lab3.2.txt
ssh target cat /tmp/lab3.2.txt

### What about  making it into a playbook for next use and leverage loops in ansible

In [ ]:
# Define a play

cat > result.yml << EOF
- hosts: localhost
  gather_facts: true
  tasks:
    - name: Check result for each host 
      command: ssh "{{ item }}" cat /tmp/lab3.2.txt
      register: res
      with_items: "{{ groups['all'] }}"

    - name: Printing output
      shell: echo "{{ item.stdout }}"
      with_items: "{{ res.results }}"
      loop_control:
        label: "{{ item.stdout }}"
EOF

### Let's check again

In [ ]:
ansible-playbook -i inventory result.yml

## Conditions

Use the when condition to control whether a task or role runs or is skipped. This is normally used to change play behavior based on facts from the destination system.

In our case, we will use the when condition on the targets to modify the content of the file based on variables set upper.

In [ ]:
# Define a play

cat > play3.yml << EOF
- hosts: all
  gather_facts: true
  tasks:
    - include_vars: vars/variables_local.yml
    - name: Template a file to lab3.3.txt
      template:
        src: templates/lab3.3.txt.j2
        dest: /tmp/lab3.3.txt
        mode: '0644'
EOF

### Execute Now

In [ ]:
ansible-playbook -i inventory play3.yml

### And Check result using the result .yml play

In [ ]:
ansible-playbook -i inventory result.yml

## Summary:
In this lab, we discovered variabls and templates.
Let's now move on to the next topic: Roles and vaults.

* [Lab 4](4-WKSHP-Ansible101-Roles.ipynb)